In [1]:
import json
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
rawdata = json.load(open('modified_Data_1.json'))["data"]
liwc_ids = np.genfromtxt('liwc_Data.csv', delimiter=',',usecols=(0),dtype=None).tolist()
liwc_data = np.genfromtxt('liwc_Data_noID.csv', delimiter=',')

In [3]:
rawIds = list()
Y = list()
'''just removing "\xef\xbb\xbf" from the first ID'''
liwc_ids[0] = '966590693376781_1502441776458334'

In [4]:
'''there are 3371 rows in raw data and 2971 rows, 93 cols with messages (LIWC)
from raw data, we need dayofweek, month for X and likes for Y'''
rawX = np.zeros((len(rawdata), 95))

In [5]:
for i in range(len(rawdata)):
    postID = rawdata[i]['id']
    rawIds.append(postID)
    rawX[i][0] = rawdata[i]['dayofweek']
    rawX[i][1] = rawdata[i]['month']
    if(postID in liwc_ids) :
        rawX[i][2:95] = liwc_data[liwc_ids.index(postID)]
    likes = rawdata[i]['likes']
    if(likes < 100):
        Y.append(0)
    else:
        Y.append(1)

In [6]:
print(rawIds[0], rawX[0], Y[0])

(u'966590693376781_1502441776458334', array([  0.  ,  11.  ,   8.  ,  71.61,  99.  ,  74.76,  25.77,   8.  ,
         0.  ,  75.  ,  50.  ,  12.5 ,  12.5 ,   0.  ,   0.  ,  12.5 ,
         0.  ,   0.  ,   0.  ,  12.5 ,  12.5 ,   0.  ,  12.5 ,  12.5 ,
         0.  ,  12.5 ,   0.  ,   0.  ,  12.5 ,  25.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  12.5 ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,  12.5 ,   0.  ,   0.  ,  12.5 ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,  25.  ,   0.  ,  12.5 ,  12.5 ,  12.5 ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,  12.5 ,   0.  ,   0.  ,  12.5 ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ]), 1)


In [7]:
'''split data'''
trainIDs, testIDs, Xtrain, Xtest, ytrain, ytest = train_test_split(rawIds, rawX, Y, test_size=0.2, random_state=420)

'''standardize and normalize'''
normalizer = Normalizer()
scaler = StandardScaler()

scaler.fit(Xtrain)
normalizer.fit(Xtrain)

Xtrain = scaler.transform(Xtrain)
Xtrain = normalizer.transform(Xtrain)

Xtest = scaler.transform(Xtest)
Xtest = normalizer.transform(Xtest)


In [ ]:
mlp = MLPClassifier()
